# Pytorch의 nn.Embedding
- Pytorch의 Embedding Layer는 word2vec과 마찬가지로 word embedding vector를 찾는 **Lookup Table**이다.
	- 단어의 **정수의 고유 index**가 입력으로 들어오면 Embedding Layer의 **그 index의 Vector**를 출력한다.
	- 모델이 학습되는 동안 모델이 풀려는 문제에 맞는 값으로 Embedding Layer의 vector들이 업데이트 된다.
	- Word2Vec의 embedding vector 학습을 nn.Embedding은 자신이 포함된 모델을 학습 하는 과정에서 한다고 생각하면 된다.

In [ ]:
import torch
import torch.nn as nn

In [46]:
embed = nn.Embedding(
	num_embeddings=20_000,	# vocab size (단어 사전의 단어 수) -> 총 몇개의 단어에 대한 embedding vector를 만들지 정해줌.
	embedding_dim=200,		# embedding vector의 차원수 -> 개별 단어를 몇개의 숫자(feature)로 표현할지.
)

In [47]:
embed.weight, embed.weight.shape

(Parameter containing:
 tensor([[ 1.1805,  0.8355, -0.2973,  ...,  2.3843,  0.1475,  0.8621],
         [ 0.6521, -0.6876,  0.3673,  ...,  0.2631,  0.5408,  0.2684],
         [-0.9871,  0.5435,  0.1098,  ..., -0.0611,  1.1731, -0.6244],
         ...,
         [ 0.1002, -0.4966, -1.3286,  ..., -2.1819,  0.8691, -1.3715],
         [-0.8155, -0.8340, -1.0334,  ..., -0.3444, -1.3056,  1.3306],
         [-0.9277, -1.1484,  0.0081,  ...,  1.5903,  0.8394, -0.6785]],
        requires_grad=True),
 torch.Size([20000, 200]))

In [48]:
# embedding layer의 입력 : 문서를 구성하는 토큰들의 ID(int)를 1차원으로 묶어서 전달.

# doc = 나는:30|어제:159|밥을:9000|먹었다:326
doc = torch.tensor([[30, 158, 9000, 326],[30, 158, 9000, 326],[30, 158, 9000, 326]], dtype=torch.int64)
embedding_vector = embed(doc)
embedding_vector.shape

# [3 : batch_size, 4 : seq_len, 200 : embedding vector 차원 수]

torch.Size([3, 4, 200])

## 전처리된 dataset load

In [49]:
import pickle
# pickle로 저장한 전처리한 dataset 읽어오기
with open("datasets/nsmc/preprocessing_trainset.pkl", "rb") as fr:
	train_dict = pickle.load(fr)

with open("datasets/nsmc/preprocessing_testset.pkl", "rb") as fr:
	test_dict = pickle.load(fr)

In [50]:
train_inputs = train_dict["input"]
train_labels = train_dict["output"]
test_inputs = test_dict["input"]
test_labels = test_dict["output"]

all_inputs = train_inputs + test_inputs		# vocab 만들 때 사용

### 토큰화
- Subword 방식 토큰화 적용
- Byte Pair Encoding 방식으로 huggingface tokenizer 사용
	- BPE: 토큰을 글자 단위로 나눈뒤 가장 자주 등장하는 글자 쌍(byte paire)를 찾아 합친뒤 어휘사전에 추가한다.
	- https://huggingface.co/docs/tokenizers/quicktour
	- `pip install tokenizers`

In [51]:
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

In [ ]:
vocab_size = 30_000		# vocab의 최대 단어 수
min_frequency = 5		# 사전에 추가할 최소 빈도수
tokenizer = Tokenizer(
	BPE(unk_token="[UNK]")
)
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(
	vocab_size=vocab_size,
	min_frequency=min_frequency,
	special_tokens=["[UNK]", "[PAD]"],
	continuing_subword_prefix="##"
	# 단어의 중간에 나오는 subword일 경우 앞에 ## 붙여주기
	# "시작하는" -> "시작", "하는" => "시작","##하는"
)
tokenizer.train_from_iterator(all_inputs, trainer=trainer)	# vocab 생성 == tokenizer학습

## Dataset, DataLoader 생성

In [ ]:
from torch.utils.data import Dataset, DataLoader

class NSMCDataset(Dataset):
	def __init__(self, texts, labels, max_length, tokenizer):
		"""
		texts: list - 댓글 목록. 리스트에 댓글들을 담아서 받는다. ["댓글", "댓글", ...]
		labels: list - 댓글 감정 목록. 
		max_length: 개별 댓글의 최대 token 개수. 모든 댓글의 토큰수를 max_length에 맞춘다.(Sequence 개수를 맞춘다)
		tokenizer: Tokenizer
		"""
		self.max_length = max_length
		self.tokenizer = tokenizer
		self.labels = labels
		# self.texts : 입력 댓글 - token id로 변환된 댓글(문서), 글자 수는 max_length에 맞춤
		#			   max_length 보다 적으면 [PAD] 추가, max_length보다 많으면 잘라냄
		self.texts = [self.__pad_token_sequences(tokenizer.encode(txt).ids) for txt in texts]

	###########################################################################################
	# id로 구성된 개별 문장 tokenizer list를 받아서 패딩 추가 [20, 2, 1] => [20, 2, 1, 0, 0, 0, ..]
	# max_length에 token list의 개수를 맞춰주는 func
	############################################################################################
	def __pad_token_sequences(self, token_sequences):
		"""
		id로 구성된 개별 문서(댓글)의 token_id list를 받아서 max_length 길이에 맞추는 메소드
		max_length 보다 토큰수가 적으면 [PAD] 추가, 많으면 max_length 크기로 줄인다.
			ex) max_length = 5, [PAD] token id가 0
			- [20, 2, 1] => [20, 2, 1, 0, 0]
			- [20, 30, 40, 50, 60, 70, 80][:5] -> [20, 30, 40, 50, 60]
		"""
		pad_token_id = self.tokenizer.token_to_id("[PAD]")
		seq_len = len(token_sequences)	# 입력받은 토큰 개수.
		result = None
		if seq_len > self.max_length:		# 잘라내기
			result = token_sequences[:?]	# self.max_length까지 잘라낼려면 ? 		(정답 : self.max_length)
		else:
			result = token_sequences + ([pad_token_id] * (self.max_length - seq_len))
		return result

	def __len__(self):
		return len(self.labels)		# 총 data개수 반환

	def __getitem__(self, idx):
		"""
		idx 번째 text와 label을 학습 가능한 type으로 변환해서 반환
		Parameter
			idx: int 조회할 index
		Return
			tuple: (torch.LongTensor, torch.FloatTensor) - 댓글 토큰_id 리스트, 정답 Label
		"""
		txt = self.texts[idx]
		label = self.labels[idx]
		return (torch.tensor(txt, dtype=torch.int64), torch.tensor([label], dtype=torch.float32))

In [ ]:
MAX_LENGTH = 30
trainset = NSMCDataset(train_inputs, train_labels, MAX_LENGTH, tokenizer)
testset = NSMCDataset(test_inputs, test_labels, MAX_LENGTH, tokenizer)

In [ ]:
BATCH_SIZE = 64
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = DataLoader(testset, batch_size=BATCH_SIZE)

# 모델링
- Embedding Layer를 이용해 Word Embedding Vector를 추출한다.
- LSTM을 이용해 Feature 추출
- Linear + Sigmoid로 댓글 긍정일 확률 출력
  
![outline](figures/rnn/RNN_outline.png)

## 모델 정의

In [ ]:
import torch
import torch.nn as nn
from torchinfo import summary
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# model def
class NSMCClassifier(nn.Module):

	def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, bidireational=True, dropout_rate=0.2):
		"""
		Args:
			vocab_size(int) : 어휘사전의 총 어휘수
			embedding_dim(int) : (word) embedding vector의 차원수
			hidden_size(int) : LSTM의 hidden state의 feature 수
			num_layers(int) : LSTM의 layer의 개수
			bidireational(bool) : LSTM의 양방향 여부
			dropout_rate(float) : LSTM이 두 개 이상의 layer로 구성된 경우 적용할 dropout 비율
								  Dropout Layer의 dropout 비율
		"""
		super().__init__()
		# model을 구성할 Layer들을 정의 : Embedding, LSTM, Dropout, Linear(추론기기), Sigmoid

		self.embedding = nn.Embedding(
			num_embeddings=vocab_size,		# 총 단어(토큰)수 -> tokenizer에 등록된 총 단어 수
			embedding_dim=embedding_dim,		# embedding vector의 차원 수
			padding_idx=0					# [PAD]의 token ID(tokenizer.token_to_index("[PAD]") 가 0인 것을 아니 그냥 0으로 넣어준것)
											# padding token은 학습하지 않는다.
		)
		# embedding layer의 출력 shape : (batch_size : 64, seq_len : 문서 토큰 수, embedding_dim)

		self.lstm= nn.LSTM(
			input_size=embedding_dim,	# 개별 토큰(단어)의 feature수(embedding -> LSTM)
			hidden_size=hidden_size,
			num_layers=num_layers,
			bidirectional=bidireational,
			dropout=dropout_rate if num_layers > 1 else 0	# stacked rnn일 경우 설정.
		)

		self.dropout = nn.Dropout(dropout_rate)		# LSTM과 Linear 사이에 과적합 방지를 위해서 사용

		# LSTM의 출력 : out, (hidden, cell)
		# out : 모든 timestep의 hidden state 값 [seq_len, batch, hidden * bidirectional ]
		# hidden : 마지막 timestep의 hidden state(단기 기억)
		# cell : 마지막 timestep의 cell state(장기 기억)

		input_features = hidden_size*2 if bidireational else hidden_size
		self.classifier = nn.Linear(input_features, 1)	# 출력 1: 이진분류 -> positive의 확률
		self.sigmoid = nn.Sigmoid()			# classifier의 출력값을 확률(0 ~ 1)값으로 변환하는 func


	def forward(self, X):
		"""
		Args:
			X(tensor) : 입력 문서 토큰 list. shape : [batch_size, max_length : anstjxhzmstn] - [64, 30]
		"""
		embedding_vectors = self.embedding(X)
		# [batch, seq_len] -> embedding -> [batch_size, seq_len, embedding_dim]
		# LSTM - batch_first = False : 입력 shape - [seq_len, batch_size, embedding_dim]\
		# embedding_vectors의 batch 축과 seq_len 축(값의 위치)을 바꿔준다!
		embedding_vectors = embedding_vectors.transpose(1,0)	# 0번 축을 1번 축으로, 1번 축을 0번 축으로
		out, _ = self.lstm(embedding_vectors)
		# out.shape : [seq_len, batch_size, hidden_size * (2 if bidireational else 1)]
		# classifier(linear)에는 out의 마지막 index(마지막 seq) 값을 입력
		output = self.dropout(out[-1])
		output = self.classifier(output)
		last_output = self.sigmoid(output)

		return last_output


## 모델 생성

In [ ]:
# Model 생성 전 변수들 먼저 선언
VOCAB_SIZE = tokenizer.get_vocab_size()		# 총 어휘수
EMBEDDING_DIM = 100
HIDDEN_SIZE = 64
NUM_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT_RATE = 0.3

# model의 복잡도 올린다 -> EMBEDDING_DIM, HIDDEN_SIZE, NUM_LAYERS를 크게!
# Auto regressive model이 아니면 BIDIRECTIONAL=True (양방향)

In [ ]:
model = NSMCClassifier(
	vocab_size=VOCAB_SIZE,
	embedding_dim=EMBEDDING_DIM,
	hidden_size=HIDDEN_SIZE,
	num_layers=NUM_LAYERS,
	bidireational=BIDIRECTIONAL,
	dropout_rate=DROPOUT_RATE
)

model = model.to(device)
print(model)

In [ ]:
# summary
i = torch.randint(1, 10, (64, MAX_LENGTH))	# int64 type의 dummy input data 생성
# input shape : ...
summary(model, input_data= i, device=device)
# summary(model, input_shape) -> 내부적으로 inputdata(float32)를 생성해서 추론함.

## 학습

### Train/Test 함수 정의

In [ ]:
# 1 epoch train하는 func
def train(model, dataloader, loss_fn, optimizer, device="cpu"):
	# 1. model을 train mode로 변환
	model.?()  # 학습 모드로 전환 				(정답: train)
	# 2. model을 device로 이동
	model = model.to(device)
	total_loss= 0.0 	# step별 loss를 누적

	# step 단위로 model train (batch)
	for X, y in dataloader:
		# 1. X, y를 device로 이동
		X, y = X.to(device), y.to(device)
		# 2. predict
		pred = model(X)
		# 3. loss cal
		loss = loss_fn(pred, y)
		# 4. Gradient cal
		loss.?()  # 손실함수로부터 역전파 수행 					(정답: backward)
		# 5. parametor update (w.data - w.grad * lr)
		optimizer.?()  # 파라미터 갱신 						(정답: step)
		# 6. Gradient 초기화
		optimizer.?()  # 매 학습 반복마다 기울기 초기화 		(정답: zero_grad)
		# loss 누적
		total_loss += loss.item()
	# 1 epoch train 완료
	return total_loss / len(dataloader)		# 1 epoch의 train loss return. (total loss / step 수)

In [ ]:
# 1 epoch eval하는 func
def test(model, dataloader, loss_fn, device="cpu"):
	# 1. model을 eval mode로 변환
	model.?()  # 평가 모드로 전환 				(정답: eval)
	# 2. model을 devie로 이동
	model = model.to(device)

	#loss, accracy
	total_loss = 0.0
	total_acc = 0.0

	with torch.no_grad():
		# step 단위로 model eval
		for X, y in dataloader:
			# 1. X, y를 device로 이동
			X, y = X.to(device), y.to(device)
			# 2. predict
			pred_proba = model(X)	# 양성일 확률
			pred_label = (pred_proba > 0.5).type(torch.int32)
			total_loss += loss_fn(pred_proba, y).item()
			total_acc += (pred_label == y).sum().item()

		# loss, acc 값을 return
		return total_loss / len(dataloader) , total_acc / len(dataloader.dataset)

### Train

In [ ]:
LR = 0.0001
EPOCHS = 3
loss_fn = nn.BCELoss()
optimizer = ?  # Adam 옵티마이저 정의           (정답: torch.optim.Adam(model.parameters(), lr=0.01))

In [ ]:
import time
s = time.time()
train_loss_list = []
val_loss_list = []
val_acc_list = []
for epoch in range(EPOCHS):
	train_loss = train(model, train_loader, loss_fn, optimizer, device)
	val_loss, val_acc = test(model, test_loader, loss_fn, device)
	train_loss_list.append(train_loss)
	val_loss_list.append(val_loss)
	val_acc_list.append(val_acc)
	print(f"[{epoch}/{EPOCHS}] train loss : {train_loss}, val loss : {val_loss}, val acc : {val_acc}")
e = time.time()
print(e-s)

## 모델 save/load

In [ ]:
# save
torch.save(model, "saved_models/nsmc/model.pt")

In [ ]:
import torch
# load
load_model = torch.load("model.pt", weights_only=False)

## 전처리 함수들

In [ ]:
from konlpy.tag import Okt

morph_tokenizer = Okt()

def text_preprocessing(text):
	
	text = text.lower()
	text = re.sub(f"[{string.punctuation}]+", ' ', text)
	return ' '.join(morph_tokenizer.morphs(text, stem=True))

In [ ]:
def pad_token_sequences(token_sequences, max_length):
	"""padding 처리 메소드."""
	pad_token = tokenizer.token_to_id('[PAD]')  
	seq_length = len(token_sequences)           
	result = None
	if seq_length > max_length:                 
		result = token_sequences[:max_length]
	else:                                            
		result = token_sequences + ([pad_token] * (max_length - seq_length))
	return result

In [ ]:
def predict_data_preprocessing(text_list):
	"""
	모델에 입력할 수있는 input data를 생성
	Parameter:
		text_list: list - 추론할 댓글리스트
	Return
		torch.LongTensor - 댓글 token_id tensor
	"""

	# cleansing + 정규화 
	text_list = [text_preprocessing(txt) for txt in text_list]
	# text -> 토큰화
	token_list = [tokenizer.encode(txt).ids for txt in text_list]
	# token list의 size를 max_length에 맞추기
	token_list = [pad_token_sequences(token, MAX_LENGTH) for token in token_list]

	return torch.tensor(token_list, dtype=torch.int64)

## 추론

In [ ]:
import re, string
comment_list = ["아 진짜 재미없다.", "여기 식당 먹을만 해요", "이걸 영화라고 만들었냐?", "기대 안하고 봐서 그런지 괜찮은데.", "이걸 영화라고 만들었나?", "아! 뭐야 진짜.", "재미있는데.", "연기 짱 좋아. 한번 더 볼 의향도 있다.", "뭐 그럭저럭"]
input_tensor = predict_data_preprocessing(comment_list)
input_tensor.shape

In [ ]:
def predict(model, comment_list:list[str], input_tensor:torch.tensor, device="cpu"):
	"""
	model로 input_tensor를 추론해서 긍정/부정적인 댓글인지 출력
	출력 형식
		comment(댓글) label 확률
		"아 노잼"	   부정	 0.9	(부정일 확률)
		"꿀잼 ㅋ"	   긍정  0.87	(긍정일 확률)
	"""

	# 1. model을 eval mode로 변환
	model.?()  # 평가 모드로 전환 (정답: eval)
	# 2. model을 device로 이동
	model = model.to(device)
	# input_tensor = input_tensor.to(device)

	
	with torch.no_grad():	# 추론 과정이니까
		pred = model(input_tensor)	# shape : (batch, 1) -> Positive일 확률값
		print(pred)
		for txt, pos_proba in zip(comment_list, pred):
			label = "긍정적" if pos_proba.item() > 0.5 else "부정적"
			proba =	pos_proba.item() if pos_proba.item() > 0.5 else 1-pos_proba.item()		# 확률
			print(txt, label, round(proba, 3), sep="\t")

In [52]:
predict(load_model, comment_list, input_tensor, device)

tensor([[0.4584],
        [0.5452],
        [0.4386],
        [0.5529],
        [0.4386],
        [0.4592],
        [0.5762],
        [0.8015],
        [0.5042]])
아 진짜 재미없다.	부정적	0.542
여기 식당 먹을만 해요	긍정적	0.545
이걸 영화라고 만들었냐?	부정적	0.561
기대 안하고 봐서 그런지 괜찮은데.	긍정적	0.553
이걸 영화라고 만들었나?	부정적	0.561
아! 뭐야 진짜.	부정적	0.541
재미있는데.	긍정적	0.576
연기 짱 좋아. 한번 더 볼 의향도 있다.	긍정적	0.802
뭐 그럭저럭	긍정적	0.504


In [ ]:
print("분석하려는 댓글을 입력하세요. 종료하려면 '!quit'을 입력하세요.")
while True:
	comment = input("댓글 : ")
	if comment == "!quit":
		print("종료")
		
		break
	comment_list = [comment]
	input_tensor = predict_data_preprocessing([comment])
	predict(model, comment_list, input_tensor, device)